In [11]:
import pandas as pd
import numpy as np
import zipfile
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import datetime
import urllib.request, urllib.error
import os.path
import datetime

In [12]:
all_deaths_path = "data/sco_covid_data/custom_data/all_deaths_lad_18.csv"
covid_deaths_path = "data/sco_covid_data/custom_data/covid_deaths_lad_18.csv"
loc_deaths_all_path = "data/sco_covid_data/custom_data/deaths_by_location_all_lad_18.csv"
loc_deaths_corona_path = "data/sco_covid_data/custom_data/deaths_by_location_corona_lad_18.csv"
path = "https://www.nrscotland.gov.uk/files//statistics/covid19/"

In [108]:
data = [['S12000033', 'Aberdeen City'],
  ['S12000034', 'Aberdeenshire'],
  ['S12000041', 'Angus council'],
  ['S12000035', 'Argyll and Bute Council'],
  ['S12000036', 'Edinburgh'],
  ['S12000005', 'Clackmannanshire'],
  ['S12000006', 'Dumfries and Galloway'],
  ['S12000042', 'Dundee City Council'],
  ['S12000008', 'East Ayrshire Council'],
  ['S12000045', 'East Dunbartonshire Council'],
  ['S12000010', 'East Lothian Council'],
  ['S12000011', 'East Renfrewshire Council'],
  ['S12000014', 'Falkirk'],
  ['S12000015', 'Fife'],
  ['S12000046', 'Glasgow City'],
  ['S12000017', 'Highland Council'],
  ['S12000018', 'Inverclyde'],
  ['S12000019', 'Midlothian'],
  ['S12000020', 'Moray'],
  ['S12000013', 'Na h-Eileanan an Iar'],
  ['S12000021', 'North Ayrshire Council'],
  ['S12000044', 'North Lanarkshire'],
  ['S12000023', 'Orkney'],
  ['S12000024', 'Perth and Kinross'],
  ['S12000038', 'Renfrewshire'],
  ['S12000026', 'Scottish Borders'],
  ['S12000027', 'Shetland Islands'],
  ['S12000028', 'South Ayrshire Council'],
  ['S12000029', 'South Lanarkshire'],
  ['S12000030', 'Stirling'],
  ['S12000039', 'West Dunbartonshire Council'],
  ['S12000040', 'West Lothian']]

In [13]:
def get_week_num():
    return datetime.date.today().isocalendar()[1]

In [23]:
'''NEED TO REFACTOR THIS'''
def get_covid_deaths_df(path):
            
    # Try get get the most recent data
    try:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-1)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        df_all_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 2 - All deaths.csv'),header = 3, encoding='unicode-escape')
        
        df_covid_deaths_trimmed = df_covid_deaths.iloc[48:80, 1:get_week_num()+1]
        df_all_deaths_trimmed = df_all_deaths.iloc[50:82, 1:get_week_num()+1]

        
        return df_all_deaths_trimmed, df_covid_deaths_trimmed
    
    # If error, then get last week response
    except urllib.error.HTTPError as e:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-2)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        
        idx_start = np.where(df_covid_deaths['Unnamed: 1'] == 'Aberdeen City')[0][0]
        idx_end = idx_start + 32
        df_covid_deaths_trimmed = df_covid_deaths.iloc[idx_start:idx_end,1:get_week_num()]
        df_covid_deaths_trimmed = df_covid_deaths_trimmed.set_index('Unnamed: 1')
        
        return df_covid_deaths_trimmed

In [25]:
def get_death_locations_df(path, week):
            
    # Try get get the most recent data
    try:
        url = path + "covid-deaths-data-week-%d.zip" % week
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_location = pd.read_csv(zipfile.open('covid-deaths-data-week-%d_Table 3 - deaths by location.csv' % week), header=3, encoding='unicode-escape')
        idx_start = np.where(df_location['Unnamed: 0'] == 'Aberdeen City')[0][0]
        idx_end = idx_start + 32
        df_location_trimmed = df_location.iloc[idx_start:idx_end, :-2]

        
        return df_location_trimmed
    
    # If error, then get last week response
    except urllib.error.HTTPError as e:
        url = path + "covid-deaths-data-week-%d.zip" % week-1
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        
        df_location = pd.read_csv(zipfile.open('covid-deaths-data-week-%d_Table 3 - deaths by location.csv' % week), header=3, encoding='unicode-escape')

        idx_start = np.where(df_location['Unnamed: 0'] == 'Aberdeen City')[0][0]
        print(idx_start)
        idx_end = idx_start + 32
        df_location_trimmed = df_location.iloc[idx_start:idx_end, :-2]
        return df_location_trimmed

In [114]:
df_death_locations_trimmed = get_death_locations_df(path, 17)
df_death_locations_trimmed

,Unnamed: 0,Care\nHome,Home / Non-institution,Hospital,Other\ninstitution3,All locations,Unnamed: 6,Care\nHome.1,Home / Non-institution.1,Hospital.1,Other\ninstitution3.1,All locations.1
24,Aberdeen City,22.0,3.0,23,0.0,48,NaN,208,214,403,0.0,825
25,Aberdeenshire,38.0,9.0,28,0.0,75,NaN,289,267,445,1.0,"1,002"
26,Angus,26.0,3.0,13,0.0,42,NaN,141,167,224,0.0,532
27,Argyll and Bute,13.0,6.0,23,0.0,42,NaN,91,163,186,0.0,440
28,City of Edinburgh,116.0,13.0,88,0.0,217,NaN,605,452,748,0.0,"1,805"
29,Clackmannanshire,8.0,0.0,7,0.0,15,NaN,73,64,93,2.0,232
30,Dumfries and Galloway,7.0,1.0,26,0.0,34,NaN,151,219,371,0.0,741
31,Dundee City,36.0,9.0,40,0.0,85,NaN,162,213,350,0.0,725
32,East Ayrshire,13.0,5.0,23,0.0,41,NaN,108,191,278,0.0,577
33,East Dunbartonshire,23.0,2.0,21,0.0,46,NaN,157,124,191,0.0,472


In [42]:
df_death_locations_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 24 to 55
Data columns (total 12 columns):
Unnamed: 0                  32 non-null object
Care
Home                   32 non-null float64
Home / Non-institution      32 non-null float64
Hospital                    32 non-null object
Other
institution3          32 non-null float64
All locations               32 non-null object
Unnamed: 6                  0 non-null float64
Care
Home.1                 32 non-null object
Home / Non-institution.1    32 non-null object
Hospital.1                  32 non-null object
Other
institution3.1        32 non-null float64
All locations.1             32 non-null object
dtypes: float64(5), object(7)
memory usage: 3.1+ KB


In [103]:
def get_data_for_location(df_locations_trimmed, area_name):
    
    df_area = df_locations_trimmed.set_index('Unnamed: 0')
    df_area_solo = df_area.loc[[area_name]]
    df_area_solo2 = df_area_solo.iloc[:,0:3]
    df_transpose = df_area_solo2.transpose()
    df_transpose = df_transpose.reset_index()
    df_transpose = df_transpose.rename(columns={'index':'Country', area_name:'Value'})
    
    return df_transpose

In [123]:
lad_dict = {"S12000033":"Aberdeen City","S12000034":"Aberdeenshire","S12000041":"Angus","S12000035":"Argyll and Bute","S12000036":"City of Edinburgh","S12000005":"Clackmannanshire","S12000006":"Dumfries and Galloway","S12000042":"Dundee City","S12000008":"East Ayrshire","S12000045":"East Dunbartonshire","S12000010":"East Lothian","S12000011":"East Renfrewshire","S12000014":"Falkirk","S12000015":"Fife","S12000046":"Glasgow City","S12000017":"Highland","S12000018":"Inverclyde","S12000019":"Midlothian","S12000020":"Moray","S12000013":"Na h-Eileanan Siar","S12000021":"North Ayrshire","S12000044":"North Lanarkshire","S12000023":"Orkney Islands","S12000024":"Perth and Kinross","S12000038":"Renfrewshire","S12000026":"Scottish Borders","S12000027":"Shetland Islands","S12000028":"South Ayrshire","S12000029":"South Lanarkshire","S12000030":"Stirling","S12000039":"West Dunbartonshire","S12000040":"West Lothian"}

In [128]:
def populate_location_deaths(df_locations_trimmed, data_dict, root_path):
    
    for x in data_dict.keys():
        df_area = get_data_for_location(df_locations_trimmed, data_dict.get(x))
        path = root_path + f"{x}.csv"
        print(path)
        current_dir = os.getcwd()
        curr = os.path.dirname(os.getcwd())
        #print(curr)
        #df_area.to_csv(current_dir + path)

In [129]:
populate_location_deaths(df_death_locations_trimmed, lad_dict, "/data/lad/total_covid_deaths/%22")

/data/lad/total_covid_deaths/%22S12000033.csv
/data/lad/total_covid_deaths/%22S12000034.csv
/data/lad/total_covid_deaths/%22S12000041.csv
/data/lad/total_covid_deaths/%22S12000035.csv
/data/lad/total_covid_deaths/%22S12000036.csv
/data/lad/total_covid_deaths/%22S12000005.csv
/data/lad/total_covid_deaths/%22S12000006.csv
/data/lad/total_covid_deaths/%22S12000042.csv
/data/lad/total_covid_deaths/%22S12000008.csv
/data/lad/total_covid_deaths/%22S12000045.csv
/data/lad/total_covid_deaths/%22S12000010.csv
/data/lad/total_covid_deaths/%22S12000011.csv
/data/lad/total_covid_deaths/%22S12000014.csv
/data/lad/total_covid_deaths/%22S12000015.csv
/data/lad/total_covid_deaths/%22S12000046.csv
/data/lad/total_covid_deaths/%22S12000017.csv
/data/lad/total_covid_deaths/%22S12000018.csv
/data/lad/total_covid_deaths/%22S12000019.csv
/data/lad/total_covid_deaths/%22S12000020.csv
/data/lad/total_covid_deaths/%22S12000013.csv
/data/lad/total_covid_deaths/%22S12000021.csv
/data/lad/total_covid_deaths/%22S1

In [104]:
df_tester = get_data_for_location(df_death_locations_trimmed, 'Aberdeen City')
df_tester

Unnamed: 0,Country,Value
0,Care\nHome,22
1,Home / Non-institution,3
2,Hospital,23


In [106]:
df_tester.to_csv('tester2.csv')